In [ ]:
import boto3
import sagemaker
from sagemaker.model import Model
from sagemaker import get_execution_role
from time import gmtime, strftime

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')
account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name
role = get_execution_role()
model_name = 'roberta-go-emotions-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

#replace with the ECR image URI
ecr_image = '123456789.dkr.ecr.us-east-1.amazonaws.com/test:roberta-base-go_emotions'
instance_type = 'ml.m5.xlarge'

container = {
'Image': ecr_image
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

endpoint_config_name = 'roberta-go-emotions-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

endpoint_name = 'roberta-go-emotions-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

tags = [
    {
        'Key': 'Project',
        'Value': 'replicatecaseworksagemaker'
    }
]

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
    Tags=tags)

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

print("Endpoint created")


In [6]:
import json

content_type = "application/json"
request_body = {"text": "Am happy this works and will give give feed back right away"}

payload = json.dumps(request_body)

response = runtime_sm_client.invoke_endpoint(
EndpointName=endpoint_name,
ContentType=content_type,
Body=payload)

response_data = json.loads(response['Body'].read().decode())
result = response_data[0]

print(result)


{'label': 'joy', 'score': 0.8476278781890869}


In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)
print("Resources deleted")